In [1]:
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
import keras.backend as K

Using TensorFlow backend.


In [2]:
def mean_absolute_precision_error(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
BATCH_SIZE   = 100 

LAYER_1_SIZE = 512
LAYER_2_SIZE = 512
#LAYER_3_SIZE = 1024
#LAYER_3_SIZE = 512
LAYER_4_SIZE = 16
dropout_rate = 0.

FEATURE_TYPE = 'cont_and_pred'  # 'cont' 'pred' 'all' 'all_but_pred' 'cont_and_pred' 'all_but_state_cols'
SEED         = 7

In [4]:
test   = pd.read_pickle('dataFrames/test_OneHotEncoding_new_June14th.pkl')
train  = pd.read_pickle('dataFrames/train_OneHotEncoding_new_June14th.pkl')

continuous_cols = ['destinationLatitude', 'destinationLongitude', 'distanceKM', 'sourceLatitude', 
                   'sourceLongitude', 'taxiDurationMin', 'weight', 'source', 'destination'] 

pred_cols = ['y_avg_lgb_xgb','y_gboost','y_xgb','y_bag','y_knn','y_dec','y_lgb']

vehicle_cols = ['vehicleType_joft', 'vehicleType_khavar', 'vehicleType_tak', 'vehicleType_treili', 
                'vehicleOption_bari', 'vehicleOption_hichkodam', 'vehicleOption_kafi',
                'vehicleOption_kompressi', 'vehicleOption_labehdar','vehicleOption_mosaghaf_chadori', 
                'vehicleOption_mosaghaf_felezi', 'vehicleOption_transit_chadori', 'vehicleOption_yakhchali']

#pred_cols = [ 'y_gboost', 'y_xgb', 'y_lgb']

categorical_cols = train.columns.drop(continuous_cols + pred_cols + vehicle_cols + ['ID', 'price']).tolist()

NOM = train[categorical_cols].shape[1]
renaming_dict = dict(zip(train[categorical_cols].columns, [str(x) for x in list(range(NOM)) ]))

train_renamed = train[categorical_cols].rename(columns=renaming_dict)
test_renamed  = test[categorical_cols].rename(columns=renaming_dict)

for column in continuous_cols + pred_cols + vehicle_cols:
    train_renamed[column] = train[column]
    test_renamed[column] = test[column]
    
test_renamed['ID']   = test['ID']
train_renamed['ID'] = train['ID']
test_renamed['price'] = test['price']
train_renamed['price'] = train['price']

X_train, X_val = train_test_split(train_renamed, test_size=0.2, random_state=42)
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,vehicleOption_hichkodam,vehicleOption_kafi,vehicleOption_kompressi,vehicleOption_labehdar,vehicleOption_mosaghaf_chadori,vehicleOption_mosaghaf_felezi,vehicleOption_transit_chadori,vehicleOption_yakhchali,ID,price
6987,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,52642600134,24500000.0
42079,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,52386856932,9000000.0
36862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,19056360759,8000000.0
28387,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,54034736201,3450000.0
35474,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,64195974128,3300000.0


In [5]:
if FEATURE_TYPE=='all':
    X_input_train = X_train.drop(['ID','price'], axis=1)
    X_input_val   = X_val.drop(['ID','price'], axis=1)
    train_input   = train_renamed.drop(['ID','price'], axis=1)
    test_input   = test_renamed.drop(['ID','price'], axis=1)
elif FEATURE_TYPE=='all_but_pred':
    X_input_train = X_train.drop(['ID','price'] + pred_cols, axis=1)
    X_input_val   = X_val.drop(['ID','price'] + pred_cols, axis=1)
    train_input   = train_renamed.drop(['ID','price'] + pred_cols, axis=1)
    test_input   = test_renamed.drop(['ID','price'] + pred_cols, axis=1)
elif FEATURE_TYPE=='all_but_state_cols':
    X_input_train = X_train[continuous_cols + pred_cols + vehicle_cols]
    X_input_val   = X_val[continuous_cols + pred_cols + vehicle_cols]
    train_input   = train_renamed[continuous_cols + pred_cols + vehicle_cols]
    test_input    = test_renamed[continuous_cols + pred_cols + vehicle_cols]
elif FEATURE_TYPE=='cont_and_pred':
    X_input_train = X_train[continuous_cols + pred_cols]
    X_input_val   = X_val[continuous_cols + pred_cols]
    train_input   = train_renamed[continuous_cols + pred_cols]
    test_input    = test_renamed[continuous_cols + pred_cols]
elif FEATURE_TYPE=='pred':
    X_input_train = X_train[pred_cols]
    X_input_val   = X_val[pred_cols]
    train_input   = train_renamed[continuous_cols]
    test_input    = test_renamed[continuous_cols]
elif FEATURE_TYPE=='cont':
    X_input_train = X_train[continuous_cols]
    X_input_val   = X_val[continuous_cols]
    train_input   = train_renamed[continuous_cols]
    test_input    = test_renamed[continuous_cols]
    
INPUT_SHAPE  = X_input_train.shape[1]
y_input_train = X_train.price

seed = SEED
np.random.seed(seed)

In [6]:
def MAPE_loss(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / y_true)) * 100

In [7]:
def base_model():
    model = Sequential()
    model.add(Dense(LAYER_1_SIZE, input_dim=INPUT_SHAPE, init='lecun_uniform', activation='relu'))
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(LAYER_2_SIZE, init='lecun_uniform', activation='relu'))
    model.add(Dropout(dropout_rate)) 
    #model.add(Dense(LAYER_3_SIZE, init='normal', activation='relu'))
    #model.add(Dropout(dropout_rate))
    model.add(Dense(LAYER_4_SIZE, init='lecun_uniform', activation='relu'))
    #model.add(Dropout(dropout_rate))
    #model.add(Dense(LAYER_5_SIZE, init='normal', activation='relu'))
    model.add(Dense(1, init='lecun_uniform'))
    model.compile(loss=MAPE_loss, optimizer = 'adam')
    return model

In [ ]:
#INPUT_SHAPE  = X_input_train.shape[1]
#clf_val = KerasRegressor(build_fn=base_model,  batch_size= 100, verbose=1, epochs=250)
#clf_val.fit(X_input_train,y_input_train)
#preds_val = clf_val.predict(X_input_val)
#score = mean_absolute_precision_error(preds_val, X_val.price)
#print('%.2f' % score)

# Final

In [10]:
INPUT_SHAPE  = train_input.shape[1]
clf = KerasRegressor(build_fn=base_model, epochs= 250, batch_size=100, verbose=1)
clf.fit(train_input,train_renamed.price)

Epoch 1/250
49534/49534 [==============================] - 3s 70us/step - loss: 16.8502
Epoch 2/250
49534/49534 [==============================] - 3s 60us/step - loss: 16.3198
Epoch 3/250
49534/49534 [==============================] - 3s 58us/step - loss: 16.1644
Epoch 4/250
49534/49534 [==============================] - 3s 58us/step - loss: 16.0739
Epoch 5/250
49534/49534 [==============================] - 3s 58us/step - loss: 16.0677
Epoch 6/250
49534/49534 [==============================] - 3s 58us/step - loss: 16.0508
Epoch 7/250
49534/49534 [==============================] - 3s 59us/step - loss: 15.9839
Epoch 8/250
49534/49534 [==============================] - 3s 58us/step - loss: 16.0016
Epoch 9/250
49534/49534 [==============================] - 3s 58us/step - loss: 15.9969
Epoch 10/250
49534/49534 [==============================] - 3s 58us/step - loss: 15.9625
Epoch 11/250
49534/49534 [==============================] - 3s 59us/step - loss: 15.9454
Epoch 12/250
49534/49534 [====

49534/49534 [==============================] - 3s 57us/step - loss: 15.6159
Epoch 94/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.6476
Epoch 95/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.6378
Epoch 96/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.6182
Epoch 97/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.6381
Epoch 98/250
49534/49534 [==============================] - 3s 58us/step - loss: 15.6215
Epoch 99/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.6523
Epoch 100/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.6372
Epoch 101/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.6151
Epoch 102/250
49534/49534 [==============================] - 3s 56us/step - loss: 15.6223
Epoch 103/250
49534/49534 [==============================] - 3s 56us/step - loss: 15.6251
Epoch 104/250
49534/49534 [===

49534/49534 [==============================] - 3s 58us/step - loss: 15.5178
Epoch 185/250
49534/49534 [==============================] - 3s 58us/step - loss: 15.5014
Epoch 186/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.5078
Epoch 187/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.5048
Epoch 188/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.4964
Epoch 189/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.5296
Epoch 190/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.5148
Epoch 191/250
49534/49534 [==============================] - 3s 58us/step - loss: 15.5174
Epoch 192/250
49534/49534 [==============================] - 3s 58us/step - loss: 15.5046
Epoch 193/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.5089
Epoch 194/250
49534/49534 [==============================] - 3s 57us/step - loss: 15.4770
Epoch 195/250
49534/4953

In [11]:
preds2 = clf.predict(X_input_val)
score  = mean_absolute_precision_error(preds2, X_val.price)
print('%.2f' % score)

9907/9907 [==============================] - 0s 17us/step
15.31


In [13]:
preds          = clf.predict(test_input)
y_preds_test   = [int(x) for x in preds]

15000/15000 [==============================] - 0s 13us/step


In [14]:
filename = "submission59.csv"
with open(filename,"w+") as outputfile:
    outputfile.write("ID,price\n")
    for i in range(len(y_preds_test)):
        outputfile.write(str(test.ID[i])+","+str(int(np.ceil(y_preds_test[i])))+"\n")

# GridSearch

In [ ]:
epochs = [10]#, 100, 150, 200, 250]  
batch_size = [ 20]#, 60, 80, 100, 128, 150, 200]

param_grid = dict(epochs=epochs, batch_size=batch_size)

clf_test = KerasRegressor(build_fn=base_model, epochs= 2, batch_size=100, verbose=1)


grid = GridSearchCV(estimator=clf_test, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(train_input, train_renamed.price) 


print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))